##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Prompting Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook contains examples of how to write and run your first prompts with the Gemini API.

In [79]:
%pip install -U -q "google-genai>=1.4.0"

Note: you may need to restart the kernel to use updated packages.


### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google import genai

GOOGLE_API_KEY = input("Enter your API key here:")
client = genai.Client(api_key=GOOGLE_API_KEY)

## Select your model

Now select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](./Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).

In [84]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite","gemini-2.5-flash","gemini-2.5-pro","gemini-2.0-flash"] {"allow-input":true, isTemplate: true}

## Run your first prompt

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to generate_content, and use the `.text` property to get the text content of the response.

In [88]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Give me python code to sort a list"
)

display(Markdown(response.text))

Python provides two main ways to sort lists:

1.  **`list.sort()`**: This method sorts the list **in-place**, meaning it modifies the original list and returns `None`.
2.  **`sorted()` (built-in function)**: This function returns a **new sorted list**, leaving the original list unchanged. It can also sort any iterable (like tuples, strings, etc.), always returning a list.

Let's look at examples for both, along with common sorting options.

---

### 1. `list.sort()` - Sorts In-Place

This method is available directly on list objects.

```python
print("--- Using list.sort() ---")

# Original list
my_list = [3, 1, 4, 1, 5, 9, 2, 6]
print(f"Original list: {my_list}")

# Sort in ascending order (default)
my_list.sort()
print(f"Sorted ascending (in-place): {my_list}")

# Sort in descending order
my_list.sort(reverse=True)
print(f"Sorted descending (in-place): {my_list}")

# --- Custom sorting using 'key' ---
words = ["banana", "Apple", "kiwi", "Grapefruit"]
print(f"\nOriginal words: {words}")

# Sort by string length
words.sort(key=len)
print(f"Sorted by length: {words}")

# Sort case-insensitively (e.g., 'Apple' comes before 'banana')
words.sort(key=str.lower)
print(f"Sorted case-insensitively: {words}")

# Sort a list of dictionaries by a specific key (e.g., 'age')
people = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25},
    {"name": "Charlie", "age": 35}
]
print(f"\nOriginal people: {people}")

# Sort by 'age'
people.sort(key=lambda person: person["age"])
print(f"Sorted by age: {people}")

# Sort by 'name' in reverse order
people.sort(key=lambda person: person["name"], reverse=True)
print(f"Sorted by name (reverse): {people}")
```

**Key characteristics of `list.sort()`:**
*   Modifies the list directly.
*   Returns `None` (so you can't do `new_list = my_list.sort()`).
*   Generally slightly more efficient for large lists if you don't need the original.

---

### 2. `sorted()` - Returns a New Sorted List

This built-in function is more flexible as it can take any iterable and always returns a *new* list, leaving the original untouched.

```python
print("\n--- Using sorted() ---")

# Original list
my_list_original = [3, 1, 4, 1, 5, 9, 2, 6]
print(f"Original list: {my_list_original}")

# Sort in ascending order (default)
new_list_asc = sorted(my_list_original)
print(f"New list (ascending): {new_list_asc}")
print(f"Original list after sorted(): {my_list_original}") # Original is unchanged!

# Sort in descending order
new_list_desc = sorted(my_list_original, reverse=True)
print(f"New list (descending): {new_list_desc}")

# --- Custom sorting using 'key' ---
words_original = ["banana", "Apple", "kiwi", "Grapefruit"]
print(f"\nOriginal words: {words_original}")

# Sort by string length
words_by_length = sorted(words_original, key=len)
print(f"New words (by length): {words_by_length}")

# Sort case-insensitively
words_case_insensitive = sorted(words_original, key=str.lower)
print(f"New words (case-insensitive): {words_case_insensitive}")

# Sort a list of dictionaries by a specific key (e.g., 'age')
people_original = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25},
    {"name": "Charlie", "age": 35}
]
print(f"\nOriginal people: {people_original}")

# Sort by 'age'
people_by_age = sorted(people_original, key=lambda person: person["age"])
print(f"New people (by age): {people_by_age}")

# Sort a tuple (sorted() works on any iterable, returns a list)
my_tuple = (5, 2, 8, 1, 9)
sorted_tuple = sorted(my_tuple)
print(f"\nOriginal tuple: {my_tuple}")
print(f"Sorted tuple (as list): {sorted_tuple}")
```

**Key characteristics of `sorted()`:**
*   Returns a *new* list, leaving the original iterable untouched.
*   Can sort any iterable (lists, tuples, sets, strings, etc.), but *always* returns a list.
*   Useful when you need both the original and a sorted version.

---

### When to choose which:

*   **Use `list.sort()`** when you:
    *   Don't need the original order of the list.
    *   Are working with a large list and memory efficiency is critical (avoiding creating a copy).
*   **Use `sorted()`** when you:
    *   Need to preserve the original list.
    *   Want to sort an iterable that isn't a list (like a tuple or set).
    *   Prefer a functional programming style where you transform data without side effects.

Both methods use Python's highly optimized **Timsort** algorithm, which is a hybrid stable sorting algorithm efficient for many kinds of real-world data.

# Data

## Database Integration

In [91]:
import numpy as np
import pandas as pd
import os
import sys

sys.path.append('../../')

from Database.db import SessionLocal
from Database_Table import Inventory, Order

def getDbContent():
    session = SessionLocal()
    inventory_records = session.query(Inventory).all()
    order_records = session.query(Order).all()
    session.close()
    return inventory_records, order_records

inventory, order = getDbContent()

2025-08-17 19:06:09,697 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-17 19:06:09,698 INFO sqlalchemy.engine.Engine SELECT `Inventory`.`ItemId` AS `Inventory_ItemId`, `Inventory`.`ItemName` AS `Inventory_ItemName`, `Inventory`.`ItemCategory` AS `Inventory_ItemCategory`, `Inventory`.`ItemQuantity` AS `Inventory_ItemQuantity`, `Inventory`.`UnitsSold` AS `Inventory_UnitsSold`, `Inventory`.`Weight` AS `Inventory_Weight`, `Inventory`.`Size` AS `Inventory_Size`, `Inventory`.`Priority` AS `Inventory_Priority`, `Inventory`.`Location` AS `Inventory_Location`, `Inventory`.`Date` AS `Inventory_Date`, `Inventory`.`Dispose` AS `Inventory_Dispose` 
FROM `Inventory`
2025-08-17 19:06:09,698 INFO sqlalchemy.engine.Engine [cached since 2005s ago] {}
2025-08-17 19:06:09,699 INFO sqlalchemy.engine.Engine SELECT `Order`.`OrderId` AS `Order_OrderId`, `Order`.`ItemId` AS `Order_ItemId`, `Order`.`OrderQuantity` AS `Order_OrderQuantity`, `Order`.`Sales` AS `Order_Sales`, `Order`.`Price` AS `Order_Pric

In [92]:
inventory[0].ItemName

'Laptop'

In [93]:
def dbtoList(records):
    output_list = []
    for r in records:
        if isinstance(r, Inventory):
            data = {
                "ItemId": r.ItemId, 
                "ItemName": r.ItemName,
                "Category": r.ItemCategory,
                "Quantity": r.ItemQuantity, 
                "UnitsSold": r.UnitsSold,
                "Weight": r.Weight, 
                "Size": r.Size,
                "Priority": r.Priority, 
                "Location": r.Location,
                "Date": r.Date, 
                "Dispose": r.Dispose                
            }
        elif isinstance(r, Order):
            data = {
                "OrderId": r.OrderId,
                "ItemId": r.ItemId, 
                "OrderQuantity": r.OrderQuantity, 
                "Sales": r.Sales, 
                "Price": r.Price, 
                "Discount": r.Discount,
                "Profit": r.Profit, 
                "DateOrdered": r.DateOrdered,
                "DateReceived": r.DateReceived,
                "CustomerSegment": r.CustomerSegment
            }
        else:
            continue
        output_list.append(data)
    
    return output_list

inventoryData = dbtoList(inventory)
orderData = dbtoList(order)

## Supervised Models

In [127]:
import pickle
import joblib
from datetime import datetime

class Supervised_Models:
    # Location Prediction Model
    def predict_location(input_data):
        with open('../../Supervised models/Samuel/storage_prediction_model.pkl', 'rb') as file:
            storage_prediction_model = pickle.load(file)
            
        categorical_features = {
            'Priority': ['High','Low','Medium'],
            'Product_Type': ['Clothing','Technology','Other','Sports and Fitness'],
            'Size': ['Large','Medium','Small']
        }
        numerical_features = ['Order_Quantity', 'Weight']
        one_hot_columns = []
        
        for feature, values in categorical_features.items():
            for value in values:
                one_hot_columns.append(f"{feature}_{value}")
            
        # Combine with numerical features to get all feature names
        all_feature_names = one_hot_columns + numerical_features

        features_dict = {col: 0 for col in all_feature_names}
    
        # Set one-hot encoded features
        for feature, values in categorical_features.items():
            if feature in input_data:
                selected_value = input_data[feature]
                one_hot_col = f"{feature}_{selected_value}"
                if one_hot_col in features_dict:
                    features_dict[one_hot_col] = 1
    
        # Set numerical features
        for feature in numerical_features:
            if feature in input_data:
                features_dict[feature] = float(input_data[feature])
        
        # Convert to array in the correct order
        features_array = np.array([features_dict[col] for col in all_feature_names]).reshape(1, -1)

        prediction = storage_prediction_model.predict(features_array)
        return prediction

    def demand_forecast_preprocessor(order_data, inventory_data):
        # Create Dataframe
        order = pd.DataFrame(order_data)
        inventory = pd.DataFrame(inventory_data)

        # Ensure dates are in datetime format
        order['DateOrdered'] = pd.to_datetime(order['DateOrdered'])
        
        # Extract Month (period or string, depending on preference)
        order['OrderMonth'] = order['DateOrdered'].dt.to_period('M')
        
        # Merge with Category lookup table
        merged_df = order.merge(inventory, on='ItemId', how='left')
        
        # Group and aggregate
        result_df = (
            merged_df
            .groupby(['OrderMonth', 'Category', 'CustomerSegment'], as_index=False)
            .agg(
                AveragePrice=('Price', 'mean'),
                AverageDiscount=('Discount', 'mean')
            )
        )
        
        return result_df

    # Demand forecast model
    def predict_demand_forecast(input_data):
        demand_forecast_model = joblib.load('../../Supervised models/ShernFai/model/salesforecast(categories).pkl')
        with open('../../Supervised models/ShernFai/model/salesforecast_preprocessor.pkl', 'rb') as f:
            preprocessor_data = pickle.load(f)

        categories = {
            "Clothing" : [
                "Cleats",
                "Men's Footwear",
                "Women's Apparel"
            ],
            "Technology": [
                "Electronics",
                "Video Games",
                "Cameras",
                "Computers",
            ],
            "Sports and Fitness": [
                "Cardio Equipment",
                "Indoor/Outdoor Games",
                "Water Sports",
                "Shop By Sport",
                "Camping & Hiking",
                "Fishing"
            ],
            "Other": [
                "Garden",
                "Pet Supplies"
            ]
        }

        cat_keys = list(categories.keys())

        # Extract preprocessor components
        le_category = preprocessor_data['label_encoder_category']
        reference_date = preprocessor_data['reference_date']
        unique_categories = preprocessor_data['unique_categories']
        feature_columns = preprocessor_data['feature_columns']

        # Get data
        category_name = input_data['category']
        future_month = input_data['month']
        avg_price = float(input_data['avg_price'])
        customer_segment = input_data['customer_segment']
        discount_rate = float(input_data['discount_rate'])
        
        # Parse the future date
        future_date = pd.to_datetime(future_month)
        
        # Calculate time features for the future date
        months_since_start = ((future_date - reference_date).days / 30.44)
        
        # Create test data with numerical time features
        test_data = {
            'Category Name': category_name,
            'Average Product Price': avg_price,
            'Customer Segment': customer_segment,
            'Order Item Discount Rate': discount_rate,
            # Time features (numerical - can handle ANY future date!)
            'Year': future_date.year,
            'Month': future_date.month,
            'Quarter': future_date.quarter,
            'Months_Since_Start': int(months_since_start),
            'Month_Sin': np.sin(2 * np.pi * future_date.month / 12),
            'Month_Cos': np.cos(2 * np.pi * future_date.month / 12),
            'Year_Trend': future_date.year - reference_date.year
        }
        
        # Create DataFrame
        test_df = pd.DataFrame([test_data])
        
        # Handle unknown category
        if category_name not in cat_keys:
            print(f"Unknown category '{category_name}' - using default: {cat_keys[0]}")
            test_df['Category Name'] = cat_keys[0]
            category_name = cat_keys[0]
        
        # One-hot encode customer segment
        test_df = pd.get_dummies(test_df, columns=['Customer Segment'], drop_first=True)
        
        # Ensure same columns as training (crucial!)
        test_df = test_df.reindex(columns=feature_columns, fill_value=0)
        
        # Make prediction
        total = 0
        num = len(categories[category_name])
        for subclass in categories[category_name]:
            test_df['Category Name'] = subclass
            test_df['Category Name'] = le_category.transform(test_df['Category Name'])
            total += demand_forecast_model.predict(test_df)
        
        avg_demand = total / num
        
        return avg_demand

    def detect_anomalies(inventory_list):
        anomalies_detected = []
        for item in inventory_list:
            current_location = item['Location']
            predicted_location = Supervised_Models.predict_location({
                "Priority": item['Priority'],
                "Product_Type": item['Category'],
                "Size": item['Size'],
                "Order_Quantity": item['Quantity'],
                "Weight": item['Weight']
            })[0]

            #print(f"\nCurrent Location: {current_location}")
            #print(f"Predicted Location: {predicted_location}")
            if current_location != predicted_location:
                #print(f"Anomaly detected! Item id:{item['ItemId']} is stored at location {current_location} while it should be stored at {predicted_location}.")
                anomalies_detected.append({'ItemId': item['ItemId'], 'ItemName': item['ItemName'], 'CurrentLocation': current_location, 'PredictedLocation': predicted_location})

        return anomalies_detected

s = Supervised_Models

In [119]:
print(inventoryData)

[{'ItemId': 1, 'ItemName': 'Laptop', 'Category': 'Technology', 'Quantity': 100, 'UnitsSold': 50, 'Weight': 1.5, 'Size': 10.0, 'Priority': 'High', 'Location': 'A-1', 'Date': '2025-06-01', 'Dispose': False}, {'ItemId': 2, 'ItemName': 'Chair', 'Category': 'Other', 'Quantity': 200, 'UnitsSold': 100, 'Weight': 2.0, 'Size': 20.0, 'Priority': 'Medium', 'Location': 'B-2', 'Date': '2025-07-01', 'Dispose': False}, {'ItemId': 3, 'ItemName': 'Laptop', 'Category': 'Technology', 'Quantity': 100, 'UnitsSold': 50, 'Weight': 1.5, 'Size': 10.0, 'Priority': 'High', 'Location': 'A-1', 'Date': '2025-06-01', 'Dispose': False}, {'ItemId': 4, 'ItemName': 'Chair', 'Category': 'Other', 'Quantity': 200, 'UnitsSold': 100, 'Weight': 2.0, 'Size': 20.0, 'Priority': 'Medium', 'Location': 'B-2', 'Date': '2025-07-01', 'Dispose': False}, {'ItemId': 5, 'ItemName': 'Laptop', 'Category': 'Technology', 'Quantity': 100, 'UnitsSold': 50, 'Weight': 1.5, 'Size': 10.0, 'Priority': 'High', 'Location': 'A-1', 'Date': '2025-06-01',

In [121]:
pd.DataFrame(inventoryData)

,ItemId,ItemName,Category,Quantity,UnitsSold,Weight,Size,Priority,Location,Date,Dispose
0,1,Laptop,Technology,100,50,1.5,10.0,High,A-1,2025-06-01,False
1,2,Chair,Other,200,100,2.0,20.0,Medium,B-2,2025-07-01,False
2,3,Laptop,Technology,100,50,1.5,10.0,High,A-1,2025-06-01,False
3,4,Chair,Other,200,100,2.0,20.0,Medium,B-2,2025-07-01,False
4,5,Laptop,Technology,100,50,1.5,10.0,High,A-1,2025-06-01,False
5,6,Chair,Other,200,100,2.0,20.0,Medium,B-2,2025-07-01,False
6,7,Laptop,Technology,100,50,1.5,10.0,High,A-1,2025-06-01,False
7,8,Chair,Other,200,100,2.0,20.0,Medium,B-2,2025-07-01,False
8,9,None,Technology,100,50,1.5,10.0,High,A-1,2025-06-01,False
9,10,None,Other,200,100,2.0,20.0,Medium,B-2,2025-07-01,False


In [125]:
print(s.detect_anomalies(inventoryData))

Laptop
Chair
Laptop
Chair
Laptop
Chair
Laptop
Chair
None
None
Laptop
Chair
[{'ItemId': 1, 'ItemName': 'Laptop', 'CurrentLocation': 'A-1', 'PredictedLocation': 'B-5'}, {'ItemId': 2, 'ItemName': 'Chair', 'CurrentLocation': 'B-2', 'PredictedLocation': 'B-5'}, {'ItemId': 3, 'ItemName': 'Laptop', 'CurrentLocation': 'A-1', 'PredictedLocation': 'B-5'}, {'ItemId': 4, 'ItemName': 'Chair', 'CurrentLocation': 'B-2', 'PredictedLocation': 'B-5'}, {'ItemId': 5, 'ItemName': 'Laptop', 'CurrentLocation': 'A-1', 'PredictedLocation': 'B-5'}, {'ItemId': 6, 'ItemName': 'Chair', 'CurrentLocation': 'B-2', 'PredictedLocation': 'B-5'}, {'ItemId': 7, 'ItemName': 'Laptop', 'CurrentLocation': 'A-1', 'PredictedLocation': 'B-5'}, {'ItemId': 8, 'ItemName': 'Chair', 'CurrentLocation': 'B-2', 'PredictedLocation': 'B-5'}, {'ItemId': 9, 'ItemName': None, 'CurrentLocation': 'A-1', 'PredictedLocation': 'B-5'}, {'ItemId': 10, 'ItemName': None, 'CurrentLocation': 'B-2', 'PredictedLocation': 'B-5'}, {'ItemId': 11, 'ItemName'

In [99]:
# Testing
print(s.predict_location({
    "Priority": "Medium",
    "Product_Type": "Sports and Fitness",
    "Size": "Medium",
    "Order_Quantity": 12,
    "Weight": 10.78
}))

['B-3']


In [100]:
result = s.demand_forecast_preprocessor(orderData, inventoryData)
result

,OrderMonth,Category,CustomerSegment,AveragePrice,AverageDiscount
0,2025-06,Technology,Corporate,100.0,10.0
1,2025-07,Other,Consumer,100.0,5.0


In [101]:
print(s.predict_demand_forecast({
    'category': "Clothing",
    'month': "2025-05",
    'avg_price': 10.0,
    'customer_segment': "Consumer",
    'discount_rate': 0.12
}))

[186.10974]


D:\Yong Meng Lee\anaconda3\envs\it3100\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Sections

## Products Overview

## Category Distribution

## Product Usage Forecast

## Sales Insights

In [ ]:
sales_data = orderData
sales_predictions = []
product_categories = ["Clothing","Technology","Sports and Fitness","Other"]
current_inventory = inventoryData
usage_probabilities = ""

sales_insights = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Generate a sales insights report that describes information for the following areas:

1. Sales Trends: Summarize sales over the past month, quarter, and year. Provide insights on which product categories are seeing the highest demand.
2. Product Performance: Analyze the best-selling products by quantity. Highlight the top 3 performing products.
3. Product Demand Forecast: Based on the historical sales and usage probability, forecast the demand for the next period.
4. Restocking or Discontinuation: Recommend which products should be restocked and which should be discontinued, based on sales trends and inventory levels.

Data:
- Historical sales data: {sales_data}
- Sales volume predictions for next month: {sales_predictions}
- Product categories: {product_categories}
- Current inventory levels: {current_inventory}
- Usage probabilities: {usage_probabilities}'''
)

display(Markdown(sales_insights.text))

## Storage Optimizations

In [135]:
location_predictions = []
for item in inventoryData:
    location_predictions.append({
        'Item Id': item['ItemId'],
        'Current Location': item['Location'],
        'Predicted Location': s.predict_location({
            'Priority': item['Priority'],
            'Product_Type': item['Category'],
            'Size': item['Size'],
            'Order_Quantity': item['Quantity'],
            'Weight': item['Weight']
        })[0]
    })

storage_optimizations = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Provide detailed storage optimization recommendations based on:

1. Current storage utilization metrics
2. Model-predicted optimal locations vs current locations
3. List of items flagged for relocation, including:
   - Current location
   - Recommended location

Data:
{inventoryData}
{location_predictions}
'''
)

display(Markdown(storage_optimizations.text))

Based on the provided data, here are detailed storage optimization recommendations:

## Storage Optimization Recommendations

This report analyzes your current storage utilization and a model's recommendations to propose strategic optimizations.

---

### 1. Current Storage Utilization Metrics

The current inventory is spread across two primary locations: `A-1` and `B-2`.

**1.1 Inventory Breakdown by Location:**

*   **Location `A-1`:**
    *   **Items Stored:** Laptop (7 units), Item ID 9 (unknown item, 1 unit).
    *   **Total Quantity:** 800 units (700 Laptops + 100 Item ID 9).
    *   **Total Estimated Weight:** 12 kg (7 * 1.5kg Laptop + 1 * 1.5kg Item ID 9).
    *   **Total Estimated Size:** 80.0 units (7 * 10.0 units Laptop + 1 * 10.0 units Item ID 9).
    *   **Priority Profile:** All items are designated 'High' priority.
    *   **Characteristics:** Houses smaller, lighter, and high-priority items.

*   **Location `B-2`:**
    *   **Items Stored:** Chair (5 units), Item ID 10 (unknown item, 1 unit).
    *   **Total Quantity:** 1200 units (1000 Chairs + 200 Item ID 10).
    *   **Total Estimated Weight:** 12 kg (5 * 2.0kg Chair + 1 * 2.0kg Item ID 10).
    *   **Total Estimated Size:** 120.0 units (5 * 20.0 units Chair + 1 * 20.0 units Item ID 10).
    *   **Priority Profile:** All items are designated 'Medium' priority.
    *   **Characteristics:** Houses larger, heavier, and medium-priority items.

**1.2 Overall Inventory Summary:**

*   **Total Unique Items:** 12 (though ItemName 'Laptop' and 'Chair' are repeated, they represent distinct inventory batches/IDs).
*   **Total Stock Quantity:** 2000 units (800 from A-1 + 1200 from B-2).
*   **Total Estimated Weight:** 24 kg.
*   **Total Estimated Size:** 200 units.
*   **Disposal Items:** No items are currently flagged for disposal (`Dispose: False` for all), meaning no immediate space can be reclaimed via disposition.

**1.3 Observations on Current State:**
*   Current storage seems to be organized by item type and priority, with smaller, high-priority items in `A-1` and larger, medium-priority items in `B-2`.
*   Data quality issue: `ItemName` is `None` for `ItemId` 9 and 10. This should be addressed for accurate inventory management.
*   The `UnitsSold` metric (e.g., 50% of Laptops and Chairs sold) indicates active inventory, suggesting these items are frequently accessed.

---

### 2. Model-Predicted Optimal Locations vs. Current Locations

The model has provided a radical but potentially highly efficient recommendation: **consolidate all inventory into a single new location, `B-5`**.

**2.1 Core Recommendation:**
The model suggests moving *every single item* from their current locations (`A-1` and `B-2`) to `B-5`. This implies `B-5` is identified as a significantly more optimal storage solution for your entire current inventory.

**2.2 Implications of the Model's Recommendation:**

*   **Centralization & Efficiency:** Moving all items to `B-5` suggests an optimization for picking efficiency, reduced travel time, or better space utilization within `B-5`. It could be a central hub, a newly optimized area, or a location with better access to shipping/receiving.
*   **Clearance of Existing Locations:** `A-1` and `B-2` would become completely empty. This frees up substantial space that can be re-purposed or de-commissioned.
*   **Streamlined Operations:** Having all active inventory in one predicted "optimal" zone could simplify inventory management, reduce errors, and accelerate order fulfillment processes.
*   **Potential Model Logic (Hypothesis):** The model likely considers factors such as:
    *   **Accessibility & Throughput:** `B-5` might offer superior access, making picking and replenishment faster, especially for high-priority items like Laptops.
    *   **Cubic Utilization:** `B-5` might be designed for more efficient stacking or racking, allowing both small (Laptops) and large (Chairs) items to be stored effectively together.
    *   **Labor Efficiency:** Consolidating reduces the need for workers to travel between disparate locations, saving time and effort.
    *   **Environmental Suitability:** `B-5` might possess optimal environmental conditions (e.g., temperature, humidity) suitable for all items, particularly sensitive electronics.

---

### 3. List of Items Flagged for Relocation

All 12 items currently in stock are recommended for relocation.

| Item Id | Item Name (Derived) | Current Location | Recommended Location |
| :------ | :------------------ | :--------------- | :------------------- |
| 1       | Laptop              | A-1              | B-5                  |
| 2       | Chair               | B-2              | B-5                  |
| 3       | Laptop              | A-1              | B-5                  |
| 4       | Chair               | B-2              | B-5                  |
| 5       | Laptop              | A-1              | B-5                  |
| 6       | Chair               | B-2              | B-5                  |
| 7       | Laptop              | A-1              | B-5                  |
| 8       | Chair               | B-2              | B-5                  |
| 9       | *Unknown Item*      | A-1              | B-5                  |
| 10      | *Unknown Item*      | B-2              | B-5                  |
| 11      | Laptop              | A-1              | B-5                  |
| 12      | Chair               | B-2              | B-5                  |

---

### Detailed Storage Optimization Recommendations

Based on the analysis, here are the detailed recommendations:

**1. Validate `B-5` Suitability and Capacity:**
    *   **Action:** Before any physical movement, thoroughly inspect `B-5`.
    *   **Checks:**
        *   **Physical Space:** Can `B-5` physically accommodate 2000 units across 200 size units and 24 kg? Consider both volume and floor space.
        *   **Weight Bearing:** Are the shelves/racks in `B-5` capable of supporting the combined weight of Laptops and Chairs?
        *   **Accessibility:** Is `B-5` easily accessible for material handling equipment (forklifts, hand trucks) for both inbound (replenishment) and outbound (picking) activities?
        *   **Environmental Control:** Does `B-5` offer suitable conditions for electronics (Laptops) and other goods?
        *   **Security:** Is `B-5` secure, especially given the high-value 'Laptop' inventory?
    *   **Outcome:** Confirm `B-5` is indeed capable of handling the entire inventory as per the model's recommendation. If not, the model's recommendation might need re-evaluation with additional constraints.

**2. Execute the Relocation to `B-5`:**
    *   **Prioritization:** Begin with high-priority items (Laptops) to ensure quick access to high-demand goods in their new optimal location.
    *   **Phased Approach:** For larger operations, consider a phased move to minimize disruption to daily operations. However, with only 12 distinct item IDs, a single, concentrated effort might be feasible.
    *   **Labeling & System Update:** Ensure all relocated items are correctly labeled with their new `B-5` location. Immediately update your Inventory Management System (IMS) to reflect the new locations to prevent picking errors.

**3. Address Data Quality:**
    *   **Action:** Immediately identify and update the `ItemName` for `ItemId` 9 and 10.
    *   **Impact:** Inaccurate or missing item names can lead to picking errors, difficulty in inventory tracking, and skew future optimization models.

**4. Re-purpose or Optimize Cleared Space (`A-1` and `B-2`):**
    *   **Action:** Once `A-1` and `B-2` are empty, evaluate their strategic use.
    *   **Options:**
        *   **Inbound Staging:** Use for temporary storage of newly arrived goods.
        *   **Outbound Staging:** For consolidated orders awaiting shipment.
        *   **Returns Processing:** Dedicated area for handling returns.
        *   **Value-Added Services (VAS):** Kitting, assembly, customization.
        *   **Seasonal/Overflow Storage:** For peak seasons or overstock that doesn't fit in primary zones.
        *   **Expansion:** Convert into additional office space, maintenance area, or a dedicated pick-and-pack zone.
        *   **Decommission:** If the space is no longer needed, consider reducing the facility footprint or renting out the area.

**5. Understand and Refine the Model's Logic:**
    *   **Action:** Work with the data science or logistics modeling team to understand the specific parameters and objectives that led the model to centralize everything in `B-5`.
    *   **Benefits:** This understanding will allow you to:
        *   Trust future recommendations more confidently.
        *   Identify if any critical business constraints (e.g., specific item handling requirements, future growth plans) were not fully incorporated.
        *   Iteratively improve the model for even better future recommendations.

**6. Monitor Performance and Adapt:**
    *   **Action:** After relocation, closely monitor key performance indicators (KPIs).
    *   **KPIs to Track:**
        *   **Picking Efficiency:** Time taken to pick orders.
        *   **Order Fulfillment Rate:** Speed of processing orders.
        *   **Space Utilization in B-5:** Ensure the space is being used effectively without becoming overly crowded.
        *   **Inventory Accuracy:** Maintain high accuracy rates in the new location.
    *   **Adaptation:** Be prepared to make minor adjustments to internal `B-5` layout or processes based on real-world performance.

By systematically implementing these recommendations, you can significantly improve your storage efficiency, operational flow, and overall warehouse performance.

## Anomalies Detected

In [129]:
anomalies_detected = client.models.generate_content(
    model=MODEL_ID,
    contents=
f'''Generate an anomalies section that lists all detected storage anomalies detected in a table. Include each item's current location, predicted location, item id, and name.
Include the type of anomaly for each item.

Data:
{s.detect_anomalies(inventoryData)}
'''
)

display(Markdown(anomalies_detected.text))

## Storage Anomalies Detected

The following items have been identified with storage anomalies. For each item, its current location does not match its predicted location. Additionally, specific items have a missing or empty name, indicating a data quality anomaly.

---

*   **Item ID:** 1
    *   **Item Name:** Laptop
    *   **Current Location:** A-1
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):** Location Mismatch (Current: 'A-1', Predicted: 'B-5')

*   **Item ID:** 2
    *   **Item Name:** Chair
    *   **Current Location:** B-2
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):** Location Mismatch (Current: 'B-2', Predicted: 'B-5')

*   **Item ID:** 3
    *   **Item Name:** Laptop
    *   **Current Location:** A-1
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):** Location Mismatch (Current: 'A-1', Predicted: 'B-5')

*   **Item ID:** 4
    *   **Item Name:** Chair
    *   **Current Location:** B-2
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):** Location Mismatch (Current: 'B-2', Predicted: 'B-5')

*   **Item ID:** 5
    *   **Item Name:** Laptop
    *   **Current Location:** A-1
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):** Location Mismatch (Current: 'A-1', Predicted: 'B-5')

*   **Item ID:** 6
    *   **Item Name:** Chair
    *   **Current Location:** B-2
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):** Location Mismatch (Current: 'B-2', Predicted: 'B-5')

*   **Item ID:** 7
    *   **Item Name:** Laptop
    *   **Current Location:** A-1
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):** Location Mismatch (Current: 'A-1', Predicted: 'B-5')

*   **Item ID:** 8
    *   **Item Name:** Chair
    *   **Current Location:** B-2
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):** Location Mismatch (Current: 'B-2', Predicted: 'B-5')

*   **Item ID:** 9
    *   **Item Name:** [NAME MISSING]
    *   **Current Location:** A-1
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):**
        *   Location Mismatch (Current: 'A-1', Predicted: 'B-5')
        *   Missing or Empty Item Name

*   **Item ID:** 10
    *   **Item Name:** [NAME MISSING]
    *   **Current Location:** B-2
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):**
        *   Location Mismatch (Current: 'B-2', Predicted: 'B-5')
        *   Missing or Empty Item Name

*   **Item ID:** 11
    *   **Item Name:** Laptop
    *   **Current Location:** A-1
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):** Location Mismatch (Current: 'A-1', Predicted: 'B-5')

*   **Item ID:** 12
    *   **Item Name:** Chair
    *   **Current Location:** B-2
    *   **Predicted Location:** B-5
    *   **Anomaly Type(s):** Location Mismatch (Current: 'B-2', Predicted: 'B-5')

## Summary

# PDF File Generation

## Set the temperature

Every prompt you send to the model includes parameters that control how the model generates responses. Use a `types.GenerateContentConfig` to set these, or omit it to use the defaults.

Temperature controls the degree of randomness in token selection. Use higher values for more creative responses, and lower values for more deterministic responses.

Note: Although you can set the `candidate_count` in the generation_config, 2.0 and later models will only return a single candidate at the this time.

In [14]:
from google.genai import types

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Give me a numbered list of cat facts.',
    config=types.GenerateContentConfig(
        max_output_tokens=2000,
        temperature=1.9,
        stop_sequences=['\n6'] # Limit to 5 facts.
    )
)

display(Markdown(response.text))

Here are some cat facts for you:

1.  Domestic cats spend about **70% of their day sleeping** and 15% grooming.
2.  The average lifespan of an outdoor cat is significantly shorter (2-5 years) compared to an **indoor cat (10-15+ years)**.
3.  Cats use their **whiskers** to "feel" the world around them, gauge openings, and detect changes in air currents. They are highly sensitive tactile organs.
4.  A group of cats is called a **clowder**, a group of kittens is called a kindle.
5.  Cats have a unique scent gland on their paws, which is why they **knead** — it's a way of marking territory and showing contentment.

## Learn more

There's lots more to learn!

* For more fun prompts, check out [Market a Jetpack](https://github.com/google-gemini/cookbook/blob/main/examples/Market_a_Jet_Backpack.ipynb).
* Check out the [safety quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Safety.ipynb) next to learn about the Gemini API's configurable safety settings, and what to do if your prompt is blocked.
* For lots more details on using the Python SDK, check out the [get started notebook](./Get_started.ipynb) or the [documentation's quickstart](https://ai.google.dev/tutorials/python_quickstart).